# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [187]:
problem = '''A buyer needs to acquire 239,600,480 units of a product and is considering bids from five suppliers, labeled A through E, each of whom can only supply a portion of the total required amount.
Each vendor has proposed different pricing structures, incorporating both setup fees and variable unit costs that change based on the quantity ordered.

The buyer's objective is to allocate the order among these suppliers to minimize overall costs, accounting for both setup and unit costs.

Vendor A offers a set up cost of $3855.34 and a unit cost of $61.150 per thousand of units.
Vendor A can supply up to 33 million units.

Vendor B offers a set up cost of $125,804.84 if purchasing between 22,000,000-70,000,000 units from vendor B with a unit cost of $68.099 per thousand units.
If purchasing between 70,000,001-100,000,000 units from vendor B, the set up cost increases to $269304.84 and the unit cost sinks to $66.049 per thousand units.
If purchasing between 100,000,001-150,000,000 units from vendor B, the unit cost per thousand units further decreases to $64.099, but the set up cost increases to $464304.84.
If purchasing between 150,000,001 and 160,000,000 units from vendor B, the unit cost is $62.119 per thousand units and the set up cost equals $761304.84.

Vendor C offers set up costs of $13,456.00 and a unit cost of $62.019 per thousand units.
Vendor C can supply up to 165.6 million units. Vendor D offers set up costs of $6,583.98 and a unit cost of $72.488 for a set of thousand units.

Vendor D can supply up to 12 million units at a price of $72.488 per thousand units and with a set up cost of $6583.98.

Vendor E offers free set up if purchasing between 0 and 42 million units of vendor E with a unit price of $70.150 per thousand units.
If purchasing between 42,000,001 and 77 million units from vendor E, the unit cost starts at $68.150 per thousand units, but with every one million units purchased the price decreases at a rate of 0.05 percent. An additional set up cost of $84000 will be charged as well.

Note that zero units may be purchased from vendor B: otherwise no positive number of units less than 22,000,000 may be purchased. 
'''

## 2. Generate the mathematical model

In [188]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [189]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Procurement

**Parameters:**

* **D:** Total demand (units): 239,600,480
* **M:**  A large number representing an upper bound for quantity 
* **SA:** Setup cost for Vendor A: $3855.34
* **VA:** Unit cost for Vendor A: $61.150 / thousand units
* **CAPA:** Capacity for Vendor A: 33,000,000 units
* **SB1:** Setup cost for Vendor B (22M-70M units): $125,804.84
* **VB1:** Unit cost for Vendor B (22M-70M units): $68.099 / thousand units
* **SB2:** Setup cost for Vendor B (70M-100M units): $269,304.84
* **VB2:** Unit cost for Vendor B (70M-100M units): $66.049 / thousand units
* **SB3:** Setup cost for Vendor B (100M-150M units): $464,304.84
* **VB3:** Unit cost for Vendor B (100M-150M units): $64.099 / thousand units
* **SB4:** Setup cost for Vendor B (150M-160M units): $761,304.84
* **VB4:** Unit cost for Vendor B (150M-160M units): $62.119 / thousand units
* **SC:** Setup cost for Vendor C: $13,456.00
* **VC:** Unit cost for Vendor C: $62.019 / thousand units
* **CAPC:** Capacity for Vendor C: 165,600,000 units
* **SD:** Setup cost for Vendor D: $6,583.98
* **VD:** Unit cost for Vendor D: $72.488 / thousand units
* **CAPD:** Capacity for Vendor D: 12,000,000 units
* **SE1:** Setup cost for Vendor E (0-42M units): $0
* **VE1:** Unit cost for Vendor E (0-42M units): $70.150 / thousand units
* **SE2:** Setup cost for Vendor E (42M-77M units): $84,000
* **VE2_base:** Base unit cost for Vendor E (42M-77M units): $68.150 / thousand units
* **VE2_discount:** Discount rate per million units for Vendor E (42M-77M units): 0.05%


**Decision Variables:**

* **XA:** Quantity purchased from Vendor A (units)
* **XB1:** Quantity purchased from Vendor B in the range of 22M-70M units
* **XB2:** Quantity purchased from Vendor B in the range of 70M-100M units
* **XB3:** Quantity purchased from Vendor B in the range of 100M-150M units
* **XB4:** Quantity purchased from Vendor B in the range of 150M-160M units
* **XC:** Quantity purchased from Vendor C (units)
* **XD:** Quantity purchased from Vendor D (units)
* **XE1:** Quantity purchased from Vendor E in the range of 0-42M units
* **XE2:** Quantity purchased from Vendor E in the range of 42M-77M units
* **YB1:** Binary variable, equals 1 if purchasing from Vendor B in the range of 22M-70M, 0 otherwise
* **YB2:** Binary variable, equals 1 if purchasing from Vendor B in the range of 70M-100M, 0 otherwise
* **YB3:** Binary variable, equals 1 if purchasing from Vendor B in the range of 100M-150M, 0 otherwise
* **YB4:** Binary variable, equals 1 if purchasing from Vendor B in the range of 150M-160M, 0 otherwise
* **YE2:** Binary variable, equals 1 if purchasing from Vendor E in the range of 42M-77M, 0 otherwise 

**Objective Function:**

Minimize total cost:

```
Min Z = SA + (VA * XA / 1000) + SB1 * YB1 + (VB1 * XB1 / 1000) + SB2 * YB2 + (VB2 * XB2 / 1000) + 
       SB3 * YB3 + (VB3 * XB3 / 1000) + SB4 * YB4 + (VB4 * XB4 / 1000) + SC + (VC * XC / 1000) + 
       SD + (VD * XD / 1000) + SE1 + (VE1 * XE1 / 1000) + SE2 * YE2 + (VE2_base * XE2 / 1000) -
       (VE2_discount * XE2 / 1000000)
```

**Constraints:**

* **Demand constraint:** 
   XA + XB1 + XB2 + XB3 + XB4 + XC + XD + XE1 + XE2 = D

* **Capacity constraints:**
   XA <= CAPA
   XB1 + XB2 + XB3 + XB4 <= 160,000,000 
   XC <= CAPC
   XD <= CAPD
   XE1 + XE2 <= 77,000,000

* **Vendor B purchasing range constraints:**
   22,000,000 * YB1 <= XB1 <= 70,000,000 * YB1
   70,000,001 * YB2 <= XB2 <= 100,000,000 * YB2
   100,000,001 * YB3 <= XB3 <= 150,000,000 * YB3
   150,000,001 * YB4 <= XB4 <= 160,000,000 * YB4
   YB1 + YB2 + YB3 + YB4 <= 1 

* **Vendor E purchasing range constraint:**
   0 <= XE1 <= 42,000,000 
   42,000,001 * YE2 <= XE2 <= 77,000,000 * YE2

* **Non-negativity constraints:**
   XA, XB1, XB2, XB3, XB4, XC, XD, XE1, XE2 >= 0

* **Binary constraints:**
   YB1, YB2, YB3, YB4, YE2 ∈ {0, 1} 

This model allows the buyer to determine the optimal quantity to purchase from each vendor, minimizing total cost while satisfying the total demand and respecting each vendor's capacity and pricing structure. 


## 3. Generate the pyomo code

In [190]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [191]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Parameters
model.D = pyo.Param(initialize=239600480)  # Total demand
model.M = pyo.Param(initialize=10e6) # A large number representing an upper bound for quantity

# Vendor A
model.SA = pyo.Param(initialize=3855.34)  # Setup cost
model.VA = pyo.Param(initialize=61.150)  # Unit cost per thousand units
model.CAPA = pyo.Param(initialize=33000000)  # Capacity

# Vendor B
model.SB1 = pyo.Param(initialize=125804.84) # Setup cost (22M-70M units)
model.VB1 = pyo.Param(initialize=68.099) # Unit cost (22M-70M units)
model.SB2 = pyo.Param(initialize=269304.84) # Setup cost (70M-100M units)
model.VB2 = pyo.Param(initialize=66.049) # Unit cost (70M-100M units)
model.SB3 = pyo.Param(initialize=464304.84) # Setup cost (100M-150M units)
model.VB3 = pyo.Param(initialize=64.099) # Unit cost (100M-150M units)
model.SB4 = pyo.Param(initialize=761304.84) # Setup cost (150M-160M units)
model.VB4 = pyo.Param(initialize=62.119) # Unit cost (150M-160M units)

# Vendor C
model.SC = pyo.Param(initialize=13456.00)  # Setup cost
model.VC = pyo.Param(initialize=62.019)  # Unit cost per thousand units
model.CAPC = pyo.Param(initialize=165600000)  # Capacity

# Vendor D
model.SD = pyo.Param(initialize=6583.98)  # Setup cost
model.VD = pyo.Param(initialize=72.488)  # Unit cost per thousand units
model.CAPD = pyo.Param(initialize=12000000)  # Capacity

# Vendor E
model.SE1 = pyo.Param(initialize=0) # Setup cost (0-42M units)
model.VE1 = pyo.Param(initialize=70.150) # Unit cost (0-42M units)
model.SE2 = pyo.Param(initialize=84000) # Setup cost (42M-77M units)
model.VE2_base = pyo.Param(initialize=68.150) # Base unit cost (42M-77M units)
model.VE2_discount = pyo.Param(initialize=0.05) # Discount rate per million units

# Decision Variables
model.XA = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor A
model.XB1 = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor B (22M-70M units)
model.XB2 = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor B (70M-100M units)
model.XB3 = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor B (100M-150M units)
model.XB4 = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor B (150M-160M units)
model.XC = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor C
model.XD = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor D
model.XE1 = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor E (0-42M units)
model.XE2 = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor E (42M-77M units)

model.YB1 = pyo.Var(domain=pyo.Binary)  # Binary variable for Vendor B (22M-70M units)
model.YB2 = pyo.Var(domain=pyo.Binary)  # Binary variable for Vendor B (70M-100M units)
model.YB3 = pyo.Var(domain=pyo.Binary)  # Binary variable for Vendor B (100M-150M units)
model.YB4 = pyo.Var(domain=pyo.Binary)  # Binary variable for Vendor B (150M-160M units)
model.YE2 = pyo.Var(domain=pyo.Binary)  # Binary variable for Vendor E (42M-77M units)

# Objective function
def objective_rule(model):
    return model.SA + (model.VA * model.XA / 1000) + model.SB1 * model.YB1 + (model.VB1 * model.XB1 / 1000) + model.SB2 * model.YB2 + (model.VB2 * model.XB2 / 1000) + model.SB3 * model.YB3 + (model.VB3 * model.XB3 / 1000) + model.SB4 * model.YB4 + (model.VB4 * model.XB4 / 1000) + model.SC + (model.VC * model.XC / 1000) + model.SD + (model.VD * model.XD / 1000) + model.SE1 + (model.VE1 * model.XE1 / 1000) + model.SE2 * model.YE2 + (model.VE2_base * model.XE2 / 1000) - (model.VE2_discount * model.XE2 / 1000000)
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.demand_constraint = pyo.Constraint(expr=model.XA + model.XB1 + model.XB2 + model.XB3 + model.XB4 + model.XC + model.XD + model.XE1 + model.XE2 == model.D)

model.capacity_constraint_A = pyo.Constraint(expr=model.XA <= model.CAPA)
model.capacity_constraint_B = pyo.Constraint(expr=model.XB1 + model.XB2 + model.XB3 + model.XB4 <= 160000000)
model.capacity_constraint_C = pyo.Constraint(expr=model.XC <= model.CAPC)
model.capacity_constraint_D = pyo.Constraint(expr=model.XD <= model.CAPD)
model.capacity_constraint_E = pyo.Constraint(expr=model.XE1 + model.XE2 <= 77000000)

model.vendor_B_range_constraint1 = pyo.Constraint(expr=22000000 * model.YB1 <= model.XB1)
model.vendor_B_range_constraint2 = pyo.Constraint(expr=model.XB1 <= 70000000 * model.YB1)
model.vendor_B_range_constraint3 = pyo.Constraint(expr=70000001 * model.YB2 <= model.XB2)
model.vendor_B_range_constraint4 = pyo.Constraint(expr=model.XB2 <= 100000000 * model.YB2)
model.vendor_B_range_constraint5 = pyo.Constraint(expr=100000001 * model.YB3 <= model.XB3)
model.vendor_B_range_constraint6 = pyo.Constraint(expr=model.XB3 <= 150000000 * model.YB3)
model.vendor_B_range_constraint7 = pyo.Constraint(expr=150000001 * model.YB4 <= model.XB4)
model.vendor_B_range_constraint8 = pyo.Constraint(expr=model.XB4 <= 160000000 * model.YB4)
model.vendor_B_selection_constraint = pyo.Constraint(expr=model.YB1 + model.YB2 + model.YB3 + model.YB4 <= 1)

model.vendor_E_range_constraint1 = pyo.Constraint(expr=0 <= model.XE1 <= 42000000)
model.vendor_E_range_constraint2 = pyo.Constraint(expr=42000001 * model.YE2 <= model.XE2)
model.vendor_E_range_constraint3 = pyo.Constraint(expr=model.XE2 <= 77000000 * model.YE2)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the results
print("Optimal solution:")
print(f"XA = {model.XA.value}")
print(f"XB1 = {model.XB1.value}")
print(f"XB2 = {model.XB2.value}")
print(f"XB3 = {model.XB3.value}")
print(f"XB4 = {model.XB4.value}")
print(f"XC = {model.XC.value}")
print(f"XD = {model.XD.value}")
print(f"XE1 = {model.XE1.value}")
print(f"XE2 = {model.XE2.value}")
print(f"Total cost = {model.objective.expr()}")
```

This Pyomo code defines the mathematical optimization model for procurement described in the problem statement. The code includes the definition of parameters, decision variables, objective function, and constraints. The model is then solved using the GLPK solver, and the optimal solution is printed.


## 4. Input problem data and try running the generated code

In [192]:
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Parameters
model.D = pyo.Param(initialize=239600480)  # Total demand
model.M = pyo.Param(initialize=10e6) # A large number representing an upper bound for quantity

# Vendor A
model.SA = pyo.Param(initialize=3855.34)  # Setup cost
model.VA = pyo.Param(initialize=61.150)  # Unit cost per thousand units
model.CAPA = pyo.Param(initialize=33000000)  # Capacity

# Vendor B
model.SB1 = pyo.Param(initialize=125804.84) # Setup cost (22M-70M units)
model.VB1 = pyo.Param(initialize=68.099) # Unit cost (22M-70M units)
model.SB2 = pyo.Param(initialize=269304.84) # Setup cost (70M-100M units)
model.VB2 = pyo.Param(initialize=66.049) # Unit cost (70M-100M units)
model.SB3 = pyo.Param(initialize=464304.84) # Setup cost (100M-150M units)
model.VB3 = pyo.Param(initialize=64.099) # Unit cost (100M-150M units)
model.SB4 = pyo.Param(initialize=761304.84) # Setup cost (150M-160M units)
model.VB4 = pyo.Param(initialize=62.119) # Unit cost (150M-160M units)

# Vendor C
model.SC = pyo.Param(initialize=13456.00)  # Setup cost
model.VC = pyo.Param(initialize=62.019)  # Unit cost per thousand units
model.CAPC = pyo.Param(initialize=165600000)  # Capacity

# Vendor D
model.SD = pyo.Param(initialize=6583.98)  # Setup cost
model.VD = pyo.Param(initialize=72.488)  # Unit cost per thousand units
model.CAPD = pyo.Param(initialize=12000000)  # Capacity

# Vendor E
model.SE1 = pyo.Param(initialize=0) # Setup cost (0-42M units)
model.VE1 = pyo.Param(initialize=70.150) # Unit cost (0-42M units)
model.SE2 = pyo.Param(initialize=84000) # Setup cost (42M-77M units)
model.VE2_base = pyo.Param(initialize=68.150) # Base unit cost (42M-77M units)
model.VE2_discount = pyo.Param(initialize=0.05) # Discount rate per million units

# Decision Variables
model.XA = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor A
model.XB1 = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor B (22M-70M units)
model.XB2 = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor B (70M-100M units)
model.XB3 = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor B (100M-150M units)
model.XB4 = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor B (150M-160M units)
model.XC = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor C
model.XD = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor D
model.XE1 = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor E (0-42M units)
model.XE2 = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor E (42M-77M units)

model.YB1 = pyo.Var(domain=pyo.Binary)  # Binary variable for Vendor B (22M-70M units)
model.YB2 = pyo.Var(domain=pyo.Binary)  # Binary variable for Vendor B (70M-100M units)
model.YB3 = pyo.Var(domain=pyo.Binary)  # Binary variable for Vendor B (100M-150M units)
model.YB4 = pyo.Var(domain=pyo.Binary)  # Binary variable for Vendor B (150M-160M units)
model.YE2 = pyo.Var(domain=pyo.Binary)  # Binary variable for Vendor E (42M-77M units)

# Objective function
def objective_rule(model):
    return model.SA + (model.VA * model.XA / 1000) + model.SB1 * model.YB1 + (model.VB1 * model.XB1 / 1000) + model.SB2 * model.YB2 + (model.VB2 * model.XB2 / 1000) + model.SB3 * model.YB3 + (model.VB3 * model.XB3 / 1000) + model.SB4 * model.YB4 + (model.VB4 * model.XB4 / 1000) + model.SC + (model.VC * model.XC / 1000) + model.SD + (model.VD * model.XD / 1000) + model.SE1 + (model.VE1 * model.XE1 / 1000) + model.SE2 * model.YE2 + (model.VE2_base * model.XE2 / 1000) - (model.VE2_discount * model.XE2 / 1000000)
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.demand_constraint = pyo.Constraint(expr=model.XA + model.XB1 + model.XB2 + model.XB3 + model.XB4 + model.XC + model.XD + model.XE1 + model.XE2 == model.D)

model.capacity_constraint_A = pyo.Constraint(expr=model.XA <= model.CAPA)
model.capacity_constraint_B = pyo.Constraint(expr=model.XB1 + model.XB2 + model.XB3 + model.XB4 <= 160000000)
model.capacity_constraint_C = pyo.Constraint(expr=model.XC <= model.CAPC)
model.capacity_constraint_D = pyo.Constraint(expr=model.XD <= model.CAPD)
model.capacity_constraint_E = pyo.Constraint(expr=model.XE1 + model.XE2 <= 77000000)

model.vendor_B_range_constraint1 = pyo.Constraint(expr=22000000 * model.YB1 <= model.XB1)
model.vendor_B_range_constraint2 = pyo.Constraint(expr=model.XB1 <= 70000000 * model.YB1)
model.vendor_B_range_constraint3 = pyo.Constraint(expr=70000001 * model.YB2 <= model.XB2)
model.vendor_B_range_constraint4 = pyo.Constraint(expr=model.XB2 <= 100000000 * model.YB2)
model.vendor_B_range_constraint5 = pyo.Constraint(expr=100000001 * model.YB3 <= model.XB3)
model.vendor_B_range_constraint6 = pyo.Constraint(expr=model.XB3 <= 150000000 * model.YB3)
model.vendor_B_range_constraint7 = pyo.Constraint(expr=150000001 * model.YB4 <= model.XB4)
model.vendor_B_range_constraint8 = pyo.Constraint(expr=model.XB4 <= 160000000 * model.YB4)
model.vendor_B_selection_constraint = pyo.Constraint(expr=model.YB1 + model.YB2 + model.YB3 + model.YB4 <= 1)

model.vendor_E_range_constraint1 = pyo.Constraint(expr=0 <= model.XE1 <= 42000000)
model.vendor_E_range_constraint2 = pyo.Constraint(expr=42000001 * model.YE2 <= model.XE2)
model.vendor_E_range_constraint3 = pyo.Constraint(expr=model.XE2 <= 77000000 * model.YE2)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the results
print("Optimal solution:")
print(f"XA = {model.XA.value}")
print(f"XB1 = {model.XB1.value}")
print(f"XB2 = {model.XB2.value}")
print(f"XB3 = {model.XB3.value}")
print(f"XB4 = {model.XB4.value}")
print(f"XC = {model.XC.value}")
print(f"XD = {model.XD.value}")
print(f"XE1 = {model.XE1.value}")
print(f"XE2 = {model.XE2.value}")
print(f"Total cost = {model.objective.expr()}")

PyomoException: Cannot convert non-constant Pyomo expression (0  <=  XE1) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

## 5. Correct the code to verify model viability (optional)

In [3]:
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Parameters
model.D = pyo.Param(initialize=239600480)  # Total demand
model.M = pyo.Param(initialize=10e6) # A large number representing an upper bound for quantity

# Vendor A
model.SA = pyo.Param(initialize=3855.34)  # Setup cost
model.VA = pyo.Param(initialize=61.150)  # Unit cost per thousand units
model.CAPA = pyo.Param(initialize=33000000)  # Capacity

# Vendor B
model.SB1 = pyo.Param(initialize=125804.84) # Setup cost (22M-70M units)
model.VB1 = pyo.Param(initialize=68.099) # Unit cost (22M-70M units)
model.SB2 = pyo.Param(initialize=269304.84) # Setup cost (70M-100M units)
model.VB2 = pyo.Param(initialize=66.049) # Unit cost (70M-100M units)
model.SB3 = pyo.Param(initialize=464304.84) # Setup cost (100M-150M units)
model.VB3 = pyo.Param(initialize=64.099) # Unit cost (100M-150M units)
model.SB4 = pyo.Param(initialize=761304.84) # Setup cost (150M-160M units)
model.VB4 = pyo.Param(initialize=62.119) # Unit cost (150M-160M units)

# Vendor C
model.SC = pyo.Param(initialize=13456.00)  # Setup cost
model.VC = pyo.Param(initialize=62.019)  # Unit cost per thousand units
model.CAPC = pyo.Param(initialize=165600000)  # Capacity

# Vendor D
model.SD = pyo.Param(initialize=6583.98)  # Setup cost
model.VD = pyo.Param(initialize=72.488)  # Unit cost per thousand units
model.CAPD = pyo.Param(initialize=12000000)  # Capacity

# Vendor E
model.SE1 = pyo.Param(initialize=0) # Setup cost (0-42M units)
model.VE1 = pyo.Param(initialize=70.150) # Unit cost (0-42M units)
model.SE2 = pyo.Param(initialize=84000) # Setup cost (42M-77M units)
model.VE2_base = pyo.Param(initialize=68.150) # Base unit cost (42M-77M units)
model.VE2_discount = pyo.Param(initialize=0.05) # Discount rate per million units

# Decision Variables
model.XA = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor A
model.XB1 = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor B (22M-70M units)
model.XB2 = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor B (70M-100M units)
model.XB3 = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor B (100M-150M units)
model.XB4 = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor B (150M-160M units)
model.XC = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor C
model.XD = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor D
model.XE1 = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor E (0-42M units)
model.XE2 = pyo.Var(domain=pyo.NonNegativeReals)  # Quantity purchased from Vendor E (42M-77M units)

model.YB1 = pyo.Var(domain=pyo.Binary)  # Binary variable for Vendor B (22M-70M units)
model.YB2 = pyo.Var(domain=pyo.Binary)  # Binary variable for Vendor B (70M-100M units)
model.YB3 = pyo.Var(domain=pyo.Binary)  # Binary variable for Vendor B (100M-150M units)
model.YB4 = pyo.Var(domain=pyo.Binary)  # Binary variable for Vendor B (150M-160M units)
model.YE2 = pyo.Var(domain=pyo.Binary)  # Binary variable for Vendor E (42M-77M units)

# Objective function
def objective_rule(model):
    return model.SA + (model.VA * model.XA / 1000) + model.SB1 * model.YB1 + (model.VB1 * model.XB1 / 1000) + model.SB2 * model.YB2 + (model.VB2 * model.XB2 / 1000) + model.SB3 * model.YB3 + (model.VB3 * model.XB3 / 1000) + model.SB4 * model.YB4 + (model.VB4 * model.XB4 / 1000) + model.SC + (model.VC * model.XC / 1000) + model.SD + (model.VD * model.XD / 1000) + model.SE1 + (model.VE1 * model.XE1 / 1000) + model.SE2 * model.YE2 + (model.VE2_base * model.XE2 / 1000) - (model.VE2_discount * model.XE2 / 1000000)
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.demand_constraint = pyo.Constraint(expr=model.XA + model.XB1 + model.XB2 + model.XB3 + model.XB4 + model.XC + model.XD + model.XE1 + model.XE2 == model.D)

model.capacity_constraint_A = pyo.Constraint(expr=model.XA <= model.CAPA)
model.capacity_constraint_B = pyo.Constraint(expr=model.XB1 + model.XB2 + model.XB3 + model.XB4 <= 160000000)
model.capacity_constraint_C = pyo.Constraint(expr=model.XC <= model.CAPC)
model.capacity_constraint_D = pyo.Constraint(expr=model.XD <= model.CAPD)
model.capacity_constraint_E = pyo.Constraint(expr=model.XE1 + model.XE2 <= 77000000)

model.vendor_B_range_constraint1 = pyo.Constraint(expr=22000000 * model.YB1 <= model.XB1)
model.vendor_B_range_constraint2 = pyo.Constraint(expr=model.XB1 <= 70000000 * model.YB1)
model.vendor_B_range_constraint3 = pyo.Constraint(expr=70000001 * model.YB2 <= model.XB2)
model.vendor_B_range_constraint4 = pyo.Constraint(expr=model.XB2 <= 100000000 * model.YB2)
model.vendor_B_range_constraint5 = pyo.Constraint(expr=100000001 * model.YB3 <= model.XB3)
model.vendor_B_range_constraint6 = pyo.Constraint(expr=model.XB3 <= 150000000 * model.YB3)
model.vendor_B_range_constraint7 = pyo.Constraint(expr=150000001 * model.YB4 <= model.XB4)
model.vendor_B_range_constraint8 = pyo.Constraint(expr=model.XB4 <= 160000000 * model.YB4)
model.vendor_B_selection_constraint = pyo.Constraint(expr=model.YB1 + model.YB2 + model.YB3 + model.YB4 <= 1)

model.vendor_E_range_constraint1 = pyo.Constraint(expr=0 <= model.XE1)
model.vendor_E_range_constraint1_u = pyo.Constraint(expr=model.XE1 <= 42000000)
model.vendor_E_range_constraint2 = pyo.Constraint(expr=42000001 * model.YE2 <= model.XE2)
model.vendor_E_range_constraint3 = pyo.Constraint(expr=model.XE2 <= 77000000 * model.YE2)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the results
print("Optimal solution:")
print(f"XA = {model.XA.value}")
print(f"XB1 = {model.XB1.value}")
print(f"XB2 = {model.XB2.value}")
print(f"XB3 = {model.XB3.value}")
print(f"XB4 = {model.XB4.value}")
print(f"XC = {model.XC.value}")
print(f"XD = {model.XD.value}")
print(f"XE1 = {model.XE1.value}")
print(f"XE2 = {model.XE2.value}")
print(f"Total cost = {model.objective.expr()}")

Optimal solution:
XA = 33000000.0
XB1 = 0.0
XB2 = 0.0
XB3 = 0.0
XB4 = 0.0
XC = 165600000.0
XD = 0.0
XE1 = 41000480.0
XE2 = 0.0
Total cost = 15188375.392


## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [202]:
print(response.text)

## Mathematical Optimization Model for Procurement

**Parameters:**

* **D:** Total demand (units): 239,600,480
* **M:**  A large number representing an upper bound for quantity 
* **SA:** Setup cost for Vendor A: $3855.34
* **VA:** Unit cost for Vendor A: $61.150 / thousand units
* **CAPA:** Capacity for Vendor A: 33,000,000 units
* **SB1:** Setup cost for Vendor B (22M-70M units): $125,804.84
* **VB1:** Unit cost for Vendor B (22M-70M units): $68.099 / thousand units
* **SB2:** Setup cost for Vendor B (70M-100M units): $269,304.84
* **VB2:** Unit cost for Vendor B (70M-100M units): $66.049 / thousand units
* **SB3:** Setup cost for Vendor B (100M-150M units): $464,304.84
* **VB3:** Unit cost for Vendor B (100M-150M units): $64.099 / thousand units
* **SB4:** Setup cost for Vendor B (150M-160M units): $761,304.84
* **VB4:** Unit cost for Vendor B (150M-160M units): $62.119 / thousand units
* **SC:** Setup cost for Vendor C: $13,456.00
* **VC:** Unit cost for Vendor C: $62.019 / thousa

In [203]:
print(response2.text)

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Parameters
model.D = pyo.Param(initialize=239600480)  # Total demand
model.M = pyo.Param(initialize=10e6) # A large number representing an upper bound for quantity

# Vendor A
model.SA = pyo.Param(initialize=3855.34)  # Setup cost
model.VA = pyo.Param(initialize=61.150)  # Unit cost per thousand units
model.CAPA = pyo.Param(initialize=33000000)  # Capacity

# Vendor B
model.SB1 = pyo.Param(initialize=125804.84) # Setup cost (22M-70M units)
model.VB1 = pyo.Param(initialize=68.099) # Unit cost (22M-70M units)
model.SB2 = pyo.Param(initialize=269304.84) # Setup cost (70M-100M units)
model.VB2 = pyo.Param(initialize=66.049) # Unit cost (70M-100M units)
model.SB3 = pyo.Param(initialize=464304.84) # Setup cost (100M-150M units)
model.VB3 = pyo.Param(initialize=64.099) # Unit cost (100M-150M units)
model.SB4 = pyo.Param(initialize=761304.84) # Setup cost (150M-160M units)
model.VB4 = pyo.Param(initialize=